In [35]:
import numpy as np

A = np.arange(12).reshape(3,4)
B = np.arange(12).reshape(3,4)
print(A)

Amax =np.max(A,axis=1)
AmaxT = np.transpose(Amax)

print(AmaxT)

print(B/AmaxT)


[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]
[ 3  7 11]


ValueError: operands could not be broadcast together with shapes (3,4) (3,) 

In [2]:
#%matplotlib nbagg
%matplotlib inline


import pandas as pd
import time
import struct
import numpy as np
import os
import shutil
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.animation import ArtistAnimation
import seaborn as sb
import glob

#■■バイナリ変換メソッド■■


    #データ並びは以下の通り
    #    | |           |
    #    | |           |
    #    | |           |
    #    | |           |
    #    | |           |g
    #    | |           |a
    #    | |           |s
    #    | |e          |s
    # z  | |x          |u
    #    | |h          |p
    #    | |a          |p
    #    | |u          |l
    #    | |s          |y
    #    | |t          |
    #    | |           |
    #    | |           |
    #    | |           |
    #    | |           |
    #    | |           |
    #
    # # # # # # # # x # # # # # # # 
  # 
def csvoutput_u(uf,vf,wf,xmax,ymax,zmax,y):
    #rbオプションは読み込みオンリー。書き込みたい場合には'wb'
    u=open(uf,'rb').read()
    #Structモジュールのunpack関数。第一引数はフォーマット。
    #この場合は倍精度(ダブルのd)8バイトずつ読み込む。//8とはデータ全体を8で割った商を表す。第二因数はバッファー(データ))
    ua=struct.unpack('d'*(len(u)//8),u)
    #u(流れ方向)
    ua_array=np.array([ua])
    ua_array.transpose()
    ua_array=ua_array.reshape(zmax,xmax*ymax)
    
    #print(ua_array.shape)
    
    ua_arrayy = ua_array[:,int(y*xmax):int((y+1)*xmax)]
        #ua_arrayy = ua_arrayy.reshape(zmax,xmax)    
    return ua_arrayy



def csvoutput_v(uf,vf,wf,xmax,ymax,zmax,y):
    #rbオプションは読み込みオンリー。書き込みたい場合には'wb'
    v=open(vf,'rb').read()
    #Structモジュールのunpack関数。第一引数はフォーマット。
    #この場合は倍精度(ダブルのd)8バイトずつ読み込む。//8とはデータ全体を8で割った商を表す。第二因数はバッファー(データ))
    va=struct.unpack('d'*(len(v)//8),v)
    #ｖ(高さ方向)
    va_array=np.array([va])
    va_array.transpose()
    va_array=va_array.reshape(zmax,xmax*ymax)
    
    #print(va_array.shape)
    
    va_arrayy = va_array[:,int(y*xmax):int((y+1)*xmax)]
    #v.close()
    #va_arrayy = va_arrayy.reshape(zmax,xmax)
    return va_arrayy

def csvoutput_w(uf,vf,wf,xmax,ymax,zmax,y):
    #rbオプションは読み込みオンリー。書き込みたい場合には'wb'
    w=open(wf,'rb').read()
    #Structモジュールのunpack関数。第一引数はフォーマット。
    wa=struct.unpack('d'*(len(w)//8),w)
    #ｗ(幅方向)
    wa_array=np.array([wa])
    wa_array.transpose()
    wa_array=wa_array.reshape(zmax,xmax*ymax)

    wa_arrayy = wa_array[:,int(y*xmax):int((y+1)*xmax)]
    #w.close()
    #wa_arrayy = wa_arrayy.reshape(zmax,xmax)    
    return wa_arrayy

#■■info.txtからxmaxなどの情報を取得■■    
def infoReader(myPath):
    #print(myPath + '/info.txt')
    #with構文を使うことで、close処理を行わなくても済む。通常であれば

    inf = open(myPath,mode = 'r')
    text = inf.read()
    data=text.split()
    inf.close()
    
    #with open(myPath + '/info.txt', mode = "r") as inf:
    #    text=inf.read()
    #    data=text.split()
    return data

#■■etchAmountAuto■■
#タイルの大きさを指定して、基板（３６０ｘ４６０㎜）全面のタイルごと平均値を返す
def etchAmountAuto(glassArray,devz,devx):
    
    k = int(np.size(glassArray,0)//devz)
    l = int(np.size(glassArray,1)//devx)
    etchTile = np.zeros((k,l,))    
    
    for m in range(0,k):
        for n in range(0,l):            
            a=int(m*devz)
            b=int(m*devz+devz)
            c=int(n*devx)
            d=int(n*devx+devx)
            etchPartial = glassArray[a:b,c:d]
            suma=np.sum(etchPartial)
            etchTile[m][n] = suma
    return etchTile
    
#■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■
#■■■■■■■■■■■■■メインメソッド■■■■■■■■■■■■■
#■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■

if __name__=='__main__':
    
    CFD_data = 'CFD G6-G8nozzle'
    # 基板先頭位置(topEdge)を示すディレクトリパス、リスト
    ListPathTpEdge = glob.glob('./' + CFD_data + '/*')
    #info.txt読み出しのためのファイルパスリスト
    ListPathStp =glob.glob('./' + CFD_data + '/*/*/info.txt')
    #info.txtから得る情報は、xmax,ymax,zmaxの格子情報だけなので、別にリストで得る必要はなし
    pathInfo = ListPathStp[0]
    #print(pathInfo)
    rlst = os.mkdir ('./' + str(CFD_data) + 'data')
    
    ###############################ノズル情報####################################
    nozzleSize = 'G8'
    
    if nozzleSize =='G3.5':
        W = 696
    if nozzleSize =='G8':
        W = 2000
    L = 170
    H = 8
    #exPos・・排気スリット位置(切り出し位置の指定)
    exPos = 20
    #pgPos・・給気スリット位置(切り出し終端位置指定)
    pgPos = 120
    #高さ方向のスライス位置(mm)
    sliceHight = 6
 
    ###############################ガラス物理長##############################
    glassSize = 'G6'
    
    if glassSize =='G2':
        glassW = 360
        glassL = 460
    if glassSize =='G6':
        glassW = 1129
        glassL = 1500
    if glassSize =="G8":
        glassW = 1800
        glassL = 1500
    #sideEdge・・ノズル端からガラス端までの位置
    sideEdge =( W - glassW) / 2

    #info.txtから必要な情報を読み出すinfoloader
    #-----------------------------------------------------------------------------------------------------
    #メッシュ数の情報をinfo.txtから読み出すため、各ディレクトリ内の最終ステップ名のフォルダパスをリスト
    #info.txtからxmax,ymax,zmaxを読み出すが、これら情報は先頭位置に依存しないので最初のディレクトリから１回だけ読み出す
    data2 = infoReader(pathInfo)
    times = float(data2[5])
    xmax = int(data2[7])
    ymax = int(data2[8])
    zmax = int(data2[9])
    #print(xmax,ymax,zmax)
    
    #節点間距離㎜　(xは搬送方向、yは高さ方向、zは幅方向)
    dz = W / zmax
    dx = L / xmax
    dy = H / ymax
    #---------------------------------------------------------------------------------------------------
    
    #エッチング量計算単位（分割タイルサイズ㎜）の計算------------------------------------ｰｰｰｰｰｰｰｰ-ｰｰｰｰ-
    xtile = 10
    ztile = 10
    
    #タイル単位に分割したガラス全面の空のマトリックスを作成--------------------------ｰｰｰｰｰｰｰｰ-ｰｰｰｰ-ｰｰｰｰ
    #etchAmountWholeは搬送方向成分U用、etchAmountWhole_wは基板直交成分W用
    etchAmountWhole = np.zeros((int(glassW // ztile) , int(glassL // xtile)), dtype = float)
    etchAmountWhole_w = np.zeros((int(glassW // ztile) , int(glassL // xtile)), dtype = float)
    etchAmountWhole_v = np.zeros((int(glassW // ztile) , int(glassL // xtile)), dtype = float)

    print(len(ListPathTpEdge))
    print(ListPathTpEdge[157])
    
    
    #メインループ　各基板先頭位置ディレクトリListPathTpEdge直下ディレクトリ(シミュレーションの最終ステップ数)内にある
    #ダンプファイルからデータを読み出して、累積。搬送平行・直交速度成分での最終結果をcsvファイルとして書き出すｰｰｰｰｰｰｰｰｰｰｰｰ
    for i in range(len(ListPathTpEdge)-1):
        #ListPathTpEdge(CFD直下の先頭位置を示すフォルダのリスト)から、先頭位置を示すフォルダ名のみを切り出す(topEdgeとして利用)
        topEdge = int( os.path.split(ListPathTpEdge[i])[1])
        #u,v,w各速度成分データファイルへのファイルパスを取得
        ufldPath = glob.glob('./' + CFD_data + '/*/*/ufld.raw')
        vfldPath = glob.glob('./' + CFD_data + '/*/*/vfld.raw')
        wfldPath = glob.glob('./' + CFD_data + '/*/*/wfld.raw')


        print(topEdge)
        
        #print(ufldPath)
        #print(vfldPath)
        #print(wfldPath)
        
 #       print(ufldPath[i],vfldPath[i],wfldPath[i])

        #-------------------------------ｰｰｰｰｰｰｰｰ-ｰｰｰｰ--------------------------------ｰｰｰｰｰｰｰｰ-ｰｰｰｰ-
        #csvoutput(uファイルパス,vファイルパス,wファイルパス,xmax,ymax,zmax,高さ)
        #fa:指定したｙ(高さ)のx-z平面(上面図) データ。最後の引数がスライシング高Sさ指定
        #fa : u(搬送方向速度成分)　　，　faw : w(基板垂直方向速度成分)  ,  fav : v(基板幅方向速度成分)
        
        
        glassArray = np.zeros((int(glassW // dz) , int(glassL // dx)), dtype = float)
        glassArray_w = np.zeros((int(glassW // dz) , int(glassL // dx)), dtype = float)
        glassArray_v = np.zeros((int(glassW // dz) , int(glassL // dx)), dtype = float)
        
        fa = csvoutput_u(ufldPath[i], vfldPath[i], wfldPath[i] , xmax , ymax , zmax, sliceHight // dy )
        #式上のwは通常高さ方向の速度成分だが、FlowSquare上で搬送方向にvに充てられてしまったため、fawを求めるためにcsvoutput_vを利用する
        #即ちfawで高さ方向の速度成分データを得るために、csvoutput_vを利用する
        faw = csvoutput_v(ufldPath[i], vfldPath[i], wfldPath[i] , xmax , ymax , zmax, sliceHight // dy )
        #式場のvは通常幅方向の速度成分だが、FlowSquare上では高さ方向にwに充てられてしまったため、favを求めるためにcsvoutput_wを利用する
        #即ちfavで幅方向の速度成分データを得るために、csvoutput_wを利用する
        fav = csvoutput_w(ufldPath[i], vfldPath[i], wfldPath[i] , xmax , ymax , zmax, sliceHight // dy )

        #-------------------------------ｰｰｰｰｰｰｰｰ-ｰｰｰｰ--------------------------------ｰｰｰｰｰｰｰｰ-ｰｰｰｰ-
        #fb:faからガラスが覆っているエリアで、かつ給気と排気スリット間部分のデータのみ切り出し        
        # ガラス位置毎のfa（y方向のスライスデータ）からfb(ガラス直下データのみ)を切り出し、glassArrayに張り付ける
        #『ガラスがノズル先端に入った瞬間（topEdge=0）から吹き出しスリットまで（topEdge=pgPos）』
        if  0 < topEdge <= pgPos:
            fb = fa[int(sideEdge // dz) : int(sideEdge // dz + glassW // dz) , int(exPos // dx) : int(topEdge // dx)]
            fbw = faw[int(sideEdge // dz) : int(sideEdge // dz + glassW // dz) , int(exPos // dx) : int(topEdge // dx)]
            fbv = fav[int(sideEdge // dz) : int(sideEdge // dz + glassW // dz) , int(exPos // dx) : int(topEdge // dx)]

            glassArray[0 : np.size(fb,0) , 0 : np.size(fb,1)] = fb
            glassArray_w[0 : np.size(fb,0) , 0 : np.size(fb,1)] = fbw            
            glassArray_v[0 : np.size(fb,0) , 0 : np.size(fb,1)] = fbv            

            #『ガラスが排気スリットから供給スリット全域を覆っている期間（常に処理幅100㎜）』
        if pgPos < topEdge <= glassL + exPos:
            fb = fa[int(sideEdge // dz) : int(sideEdge // dz + glassW // dz) , int(exPos // dx) : int(pgPos // dx)]
            fbw = faw[int(sideEdge // dz) : int(sideEdge // dz + glassW // dz) , int(exPos // dx) : int(pgPos // dx)]
            fbv = fav[int(sideEdge // dz) : int(sideEdge // dz + glassW // dz) , int(exPos // dx) : int(pgPos // dx)]
            
            glassArray[0 : np.size(fb,0) , topEdge - pgPos : np.size(fb,1) + topEdge - pgPos] =fb
            glassArray_w[0 : np.size(fb,0) , topEdge - pgPos : np.size(fb,1) + topEdge - pgPos] =fbw
            glassArray_v[0 : np.size(fb,0) , topEdge - pgPos : np.size(fb,1) + topEdge - pgPos] =fbv
            
        #『ガラス後端が排気スリットを通過したところから、供給スリット上を通過するまで』
        if glassL+exPos < topEdge <= glassL + pgPos:
            fb = fa[int(sideEdge // dz) : int(sideEdge // dz + glassW // dz) , int(topEdge - glassL) : int(pgPos // dx)]
            fbw = faw[int(sideEdge // dz) : int(sideEdge // dz + glassW // dz) , int(topEdge - glassL) : int(pgPos // dx)]
            fbv = fav[int(sideEdge // dz) : int(sideEdge // dz + glassW // dz) , int(topEdge - glassL) : int(pgPos // dx)]
            
            glassArray[0 : np.size(fb,0) , -np.size(fb,1) : int(glassL // dx)] = fb
            glassArray_w[0 : np.size(fb,0) , -np.size(fb,1) : int(glassL // dx)] = fbw
            glassArray_v[0 : np.size(fb,0) , -np.size(fb,1) : int(glassL // dx)] = fbv
            
        #-------------------------------ｰｰｰｰｰｰｰｰ-ｰｰｰｰ--------------------------------ｰｰｰｰｰｰｰｰ-ｰｰｰｰ-
        #累積エッチング量(タイル毎のu成分和)計算etchAmountAuto（元データ,ｘ方向分割数,y方向分割数,基板先端位置）　
        etchAmount = etchAmountAuto(glassArray , ztile // dz , xtile // dx)
        etchAmountWhole = etchAmount + etchAmountWhole
    
        etchAmount_w = etchAmountAuto(glassArray_w , ztile // dz , xtile // dx)
        etchAmountWhole_w = etchAmount_w + etchAmountWhole_w
        
        etchAmount_v = etchAmountAuto(glassArray_v , ztile // dz , xtile // dx)
        etchAmountWhole_v = etchAmount_v + etchAmountWhole_v
        

        
        #グラフの表示(ax1:搬送方向成分, ax2:基板直交成分)-------------------------------ｰｰｰｰｰｰｰｰ-ｰｰｰｰ--------------
        
        fig=plt.figure()
        
    #    im1 = plt.pcolormesh(etchAmountWhole, interpolation='none',cmap='RdBu')
        im1 = plt.pcolormesh(etchAmountWhole, cmap = 'RdBu')
        fig.colorbar(im1)
    #    plt.show()
    #    fig.colorbar(im1)
        #im2 = plt.imshow(etchAmountWhole_w, interpolation='none',cmap='RdBu' )
    #    im2 = plt.pcolormesh(etchAmountWhole_w, cmap = 'RdBu')
    #    fig.colorbar(im2)
        #im3 = ax3.imshow(etchAmountWhole_v, interpolation='none',cmap='RdBu' )
        #fig1.colorbar(im3)
        #fig1.tight_layout()
        plt.pause(0.1)
        #plt.show()
        
        #ファイル出力(１)　各先頭位置での切り出しデータ----ｰｰｰｰｰ-ｰｰｰｰ--------------------------------ｰｰｰｰｰｰｰｰ-ｰｰｰｰ-
        np.savetxt('uva_arrayt.csv',fa,delimiter=',')
        #切り出したuva_arrayt_glassから任意の指定の小領域(タイル)毎の部分和を出力
        #np.savetxt('./' + str(CFD_data) + 'data/' + str(topEdge) + '_' + 'etchAmount.csv',etchAmount,delimiter = ',')
        #np.savetxt('./' + str(CFD_data) + 'data/' + str(topEdge) + '_' + 'etchAmount_w.csv',etchAmount_w,delimiter = ',')
        #np.savetxt('./' + str(CFD_data) + 'data/' + str(topEdge) + '_' + 'etchAmount_v.csv',etchAmount_v,delimiter = ',')
     
    
    
    etchAmountWhole_velo = (etchAmountWhole ** 2 + etchAmountWhole_w ** 2 + etchAmountWhole_v ** 2) ** 0.5 

    etchAmountWholeAbs = np.abs(etchAmountWhole)
    
    #etchAmountWholeMax = np.amax(np.abs(etchAmountWhole))
    
    etchAmountWhole_w_Norm = etchAmountWhole_w / np.min(etchAmountWhole_w,axis=0)
    etchAmountWholeUV = etchAmountWholeAbs * etchAmountWhole_w_Norm      
    
    #ファイル出力(２)　（１）のデータを重ね合わせた累計データの出力
    np.savetxt('./' + str(CFD_data) + 'data/' + 'etchAmountWhole.csv',etchAmountWhole,delimiter = ',')
    np.savetxt('./' + str(CFD_data) + 'data/' + 'etchAmountWhole_w.csv',etchAmountWhole_w,delimiter = ',')
    np.savetxt('./' + str(CFD_data) + 'data/' + 'etchAmountWhole_v.csv',etchAmountWhole_v,delimiter = ',')
    
    #np.savetxt('./etch/' +  'etchAmountWhole_w_Norm.csv',etchAmountWhole_w_Norm,delimiter =',')
    #np.savetxt('./etch/' +  'etchAmountWholeUV.csv',etchAmountWholeUV,delimiter =',')
    np.savetxt('./' + str(CFD_data) + 'data/'  + 'etcAmountWhole_velo.csv', etchAmountWhole_velo,delimiter =',')



KeyboardInterrupt: 

In [9]:
import numpy as np

A = np.arange([1,2,4],[3,2,1],[2,1,4])
B = np.arange([2,1,4],[2,1,4],[5,3,1])
AB = A * B
print(AB)


TypeError: unsupported operand type(s) for -: 'list' and 'list'